In [2]:
from __future__ import print_function
from gluonts.dataset.rolling_dataset import StepStrategy, generate_rolling_dataset
import argparse
import os
import logging
from sklearn.preprocessing import MinMaxScaler
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import json
import time

from pathlib import Path

from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.util import to_pandas
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

from gluonts.model.canonical import CanonicalRNNEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deepvar import DeepVAREstimator
from gluonts.model.gp_forecaster import GaussianProcessEstimator
from gluonts.model.gpvar import GPVAREstimator
from gluonts.model.lstnet import LSTNetEstimator
from gluonts.model.n_beats import NBEATSEstimator
from gluonts.model.seq2seq import MQCNNEstimator, MQRNNEstimator, RNN2QRForecaster, Seq2SeqEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.model.transformer import TransformerEstimator
from gluonts.model.wavenet import WaveNetEstimator

from gluonts.mx.block.quantile_output import QuantileOutput
from gluonts.mx.trainer import Trainer
from gluonts.mx.block.encoder import Seq2SeqEncoder

from gluonts.model.predictor import Predictor

from gluonts.model.naive_2 import Naive2Predictor
from gluonts.model.npts import NPTSPredictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from gluonts.model.seasonal_naive import SeasonalNaivePredictor

from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100

def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

def load_json(filename):
    data = []
    with open(filename, 'r') as fin:
        while True:
            line = fin.readline()
            if not line:
                break
            datai = json.loads(line)
            data.append(datai)
    return data
def parse_data(dataset):
    data = []
    for t in dataset:
        datai = {FieldName.TARGET: t['target'], FieldName.START: t['start']}
        if 'id' in t:
            datai[FieldName.ITEM_ID] = t['id']
        if 'cat' in t:
            datai[FieldName.FEAT_STATIC_CAT] = t['cat']
        if 'dynamic_feat' in t:
            datai[FieldName.FEAT_DYNAMIC_REAL] = t['dynamic_feat']
        data.append(datai)
    return data
def parse_data_ar(dataset):
    data = []
    for t in dataset:
        datai = {FieldName.TARGET: [t['target'][3][:-12],t['target'][8][:-12]], FieldName.START: t['start']}
        if 'id' in t:
            datai[FieldName.ITEM_ID] = t['id']
        if 'cat' in t:
            datai[FieldName.FEAT_STATIC_CAT] = t['cat']
        if 'dynamic_feat' in t:
            datai[FieldName.FEAT_DYNAMIC_REAL] = t['dynamic_feat']
        data.append(datai)
    return data
def parse_data_ar_roll(dataset,start):
    data = []
    begin=int((start-pd.Timestamp("2020-01-02 00:00:00"))/pd.Timedelta("30T"))
    for t in dataset:
        datai = {FieldName.TARGET: [t['target'][3][begin:begin+30*48+1],t['target'][8][begin:begin+30*48+1]], FieldName.START: str(start)}
        if 'id' in t:
            datai[FieldName.ITEM_ID] = t['id']
        if 'cat' in t:
            datai[FieldName.FEAT_STATIC_CAT] = t['cat']
        if 'dynamic_feat' in t:
            datai[FieldName.FEAT_DYNAMIC_REAL] = t['dynamic_feat']
        data.append(datai)
    return data
def parse_data_scaling(dataset):# 把所有的销售量进行归一化
    data = []
    scaler = MinMaxScaler(feature_range=(0, 1))
    for t in dataset:
        tar=np.array(t['target'])
        tar[1] = scaler.fit_transform(tar[1,:].reshape(-1,1))[:,0]
        tar[2] = scaler.fit_transform(tar[2,:].reshape(-1,1))[:,0]
        tar[13:] = scaler.fit_transform(tar[13:,:])
        datai = {FieldName.TARGET: tar[:,:-12], FieldName.START: t['start']}# 这里放弃最后12个点，即实际训练范围到7.31日下午3点
        if 'id' in t:
            datai[FieldName.ITEM_ID] = t['id']
        if 'cat' in t:
            datai[FieldName.FEAT_STATIC_CAT] = t['cat']
        if 'dynamic_feat' in t:
            datai[FieldName.FEAT_DYNAMIC_REAL] = t['dynamic_feat']
        data.append(datai)
    return data
def parse_data_scaling2(dataset):# 把除了买一价卖一价之外的进行归一化
    data = []
    scaler = MinMaxScaler(feature_range=(0, 1))
    for t in dataset:
        tar=np.array(t['target'])
        tar[0]=scaler.fit_transform(tar[0,:].reshape(-1,1))[:,0]
        tar[1] = scaler.fit_transform(tar[1,:].reshape(-1,1))[:,0]
        tar[2] = scaler.fit_transform(tar[2,:].reshape(-1,1))[:,0]
        tar[4:8] = scaler.fit_transform(tar[4:8,:])
        tar[9:13] = scaler.fit_transform(tar[9:13,:])
        tar[13:] = scaler.fit_transform(tar[13:,:])
        datai = {FieldName.TARGET: tar[:,:-12], FieldName.START: t['start']}# 这里放弃最后12个点，即实际训练范围到7.31日下午3点
        if 'id' in t:
            datai[FieldName.ITEM_ID] = t['id']
        if 'cat' in t:
            datai[FieldName.FEAT_STATIC_CAT] = t['cat']
        if 'dynamic_feat' in t:
            datai[FieldName.FEAT_DYNAMIC_REAL] = t['dynamic_feat']
        data.append(datai)
    return data



/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [3]:
logging.basicConfig(level=logging.DEBUG)
train = load_json('./glts_train_multi_tar.json')
test = load_json('./glts_test_multi_tar.json')
freq = '30T'
import mxnet as mx
prediction_length = 1
context_length = 48*30
num_timeseries = len(train)
print('num_timeseries:', num_timeseries)
train_ds = ListDataset(parse_data_ar(train), freq=freq, one_dim_target=False)
test_ds = ListDataset(parse_data_ar(test), freq=freq, one_dim_target=False)
# train_ds = ListDataset(parse_data_scaling(train), freq=freq, one_dim_target=False)
# test_ds = ListDataset(parse_data_scaling(test), freq=freq, one_dim_target=False)
print('finish')

num_timeseries: 245
finish


In [6]:
p=os.path.join('./models', 'GPVAR_lr0.001_epoch15scaling_all_ar')
from gluonts.model.predictor import Predictor
predictor_deserialized = Predictor.deserialize(Path(p))
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,  # test dataset
    predictor=predictor_deserialized,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)
forecasts = list(forecast_it)
tss = list(ts_it)

In [4]:
print('samples_shape: ', forecasts[0].samples.shape)
print('prediction_length: ', forecasts[0].prediction_length)
forecast_entry=forecasts[18]
ts_entry = tss[18]
print(f"Number of sample paths: {forecast_entry.num_samples}")
print(f"Dimension of samples: {forecast_entry.samples.shape}")
print(f"Start date of the forecast window: {forecast_entry.start_date}")
print(f"Frequency of the time series: {forecast_entry.freq}")
print(f"Mean of the future window:\n {forecast_entry.mean}")
# print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.66)[0][[3,8]].shape}")
print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.52)}")
print(forecast_entry.quantile(0.9).shape)
print(ts_entry.iloc[-1])
print(ts_entry.index[-1])

samples_shape:  (100, 1, 2)
prediction_length:  1
Number of sample paths: 100
Dimension of samples: (100, 1, 2)
Start date of the forecast window: 2020-07-31 18:30:00
Frequency of the time series: 30T
Mean of the future window:
 [[9.615109 9.635728]]
0.5-quantile (median) of the future window:
 [[9.628544 9.633638]]
(1, 2)
0    9.60
1    9.61
Name: 2020-07-31 18:30:00, dtype: float32
2020-07-31 18:30:00


In [5]:
def cal_metrics(gt, pre , q):
    length = len(forecasts)
    mse=0
    rmse=0
    mae=0
    m_ape=0
    s_mape=0
    for i in range(length):
        ts = gt[i].to_numpy()
        p = pre[i].quantile(q)[0][np.array([3,8])]
        g = ts[-1][np.array([3,8])]
        mse += (1/length)*metrics.mean_squared_error(g, p)
        rmse += (1/length)*np.sqrt(mse)
        mae += (1/length)*metrics.mean_absolute_error(g, p)
        m_ape += (1/length)*mape(g, p)
        s_mape += (1/length)*smape(g, p)
    return mse,rmse,mae,m_ape,s_mape
# mse,rmse,mae,m_ape,s_mape=cal_metrics(tss,forecasts,0.55)
# print(mse,rmse,mae,m_ape,s_mape)

In [7]:
def cal_metrics_ar(gt, pre , q):
    length = len(forecasts)
    mse=0
    rmse=0
    mae=0
    m_ape=0
    s_mape=0
    for i in range(length):
        ts = gt[i].to_numpy()
        p = pre[i].mean[0]
        g = ts[-1]
        mse += (1/length)*metrics.mean_squared_error(g, p)
        rmse += (1/length)*np.sqrt(mse)
        mae += (1/length)*metrics.mean_absolute_error(g, p)
        m_ape += (1/length)*mape(g, p)
        s_mape += (1/length)*smape(g, p)
    return mse,rmse,mae,m_ape,s_mape
mse,rmse,mae,m_ape,s_mape=cal_metrics_ar(tss,forecasts,0.52)
print(mse,rmse,mae,m_ape,s_mape)

0.46877569057897905 0.30823287156019397 0.14813023213468196 inf 2.9273733217744358


/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide
/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide
/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide
/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide
/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide
/Users/lvhaoran/opt/anaconda3/envs/mxnet/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide


In [7]:
# pre_data=[]
# pre_data=[
#     {
#         "start":str(pd.Timestamp(tss[0].index.tolist()[-1], freq=freq)),
#         "predict0.3":val.quantile(0.3)[0].tolist(),
#         "predict0.4":val.quantile(0.4)[0].tolist(),
#         "predict0.5":val.quantile(0.5)[0].tolist(),
#         "predictmean":val.mean[0].tolist()
#     } for i, val in enumerate(forecasts)
# ]
# def write_dicts_to_file(path, data):
#     with open(path, 'wb') as fp:
#         for d in data:
#             fp.write(json.dumps(d).encode("utf-8"))
#             fp.write("\n".encode('utf-8'))
# write_dicts_to_file('./predictions_gpvar.json', pre_data)

In [8]:
total=load_json('./glts_total_multi_tar.json')
def rolling_pre(total, start, predictor=predictor_deserialized, predict_len=60):
    delta = pd.Timedelta("30T")
    start = start
    pre_res_mean = np.zeros((245, 2, predict_len))
    pre_res_01 = np.zeros((245, 2, predict_len))
    pre_res_09 = np.zeros((245, 2, predict_len))
    gt=np.zeros((245, 2, predict_len))
    for i in range(predict_len):
        print(i)
        total_data = ListDataset(parse_data_ar_roll(total,start), freq=freq, one_dim_target=False)
        forecast_it, ts_it = make_evaluation_predictions(
            dataset = total_data,  # test dataset
            predictor = predictor,  # predictor
            num_samples = 100,  # number of sample paths we want for evaluation
        )
        forecasts = list(forecast_it)
        tss = list(ts_it)
        for j in range(245):
            pre_res_mean[j,:,i] = forecasts[j].mean[0]
            pre_res_01[j,:,i] = forecasts[j].quantile(0.1)[0]
            pre_res_09[j,:,i] = forecasts[j].quantile(0.9)[0]
            gt[j,:,i] = tss[j].to_numpy()[-1]
        start+=delta
    return pre_res_mean, pre_res_01, pre_res_09, gt

In [9]:
start=pd.Timestamp("2020-07-01 17:00:00")
print(start-pd.Timestamp("2020-01-02 17:00:00"))
# ae=pd.Timestamp("2020-01-02 00:00:00")
# print(start-pd.Timedelta("30 days"))
# print((start-ae)/pd.Timedelta("30T"))
pre_res_mean, pre_res_01, pre_res_09, gt=rolling_pre(total, start)

181 days 00:00:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [10]:
np.save('./gpvar_mean.npy', pre_res_mean)
np.save('./gpvar_01.npy', pre_res_01)
np.save('./gpvar_09.npy', pre_res_09)
np.save('./gpvar_gt.npy', gt)
print('finish')

finish


In [14]:
a=np.load('./gpvar.npy')
print(a)



[[[ 3.61996918e+01  3.61951370e+01  3.61595268e+01 ... -6.99125230e-02
    8.07816759e-02 -6.28615022e-02]
  [ 3.63182220e+01  3.63438759e+01  3.62102776e+01 ... -1.68078154e-01
   -1.05007738e-01  1.30799517e-01]]

 [[ 7.30395889e+00  7.31332970e+00  7.35648870e+00 ...  5.95084066e-03
    1.66509561e-02 -1.19192638e-02]
  [ 7.33720827e+00  7.32828712e+00  7.31747818e+00 ...  1.58320661e-04
    1.32772028e-02  9.25951637e-03]]

 [[ 1.93499470e+01  1.94284248e+01  1.93419132e+01 ... -3.02987751e-02
   -4.34201136e-02 -3.75012197e-02]
  [ 1.93230343e+01  1.94272289e+01  1.93669071e+01 ...  3.09739038e-02
   -2.22518556e-02 -3.04613430e-02]]

 ...

 [[ 1.98518217e+00  1.98250127e+00  1.97858214e+00 ... -2.53176130e-03
    6.47398038e-03 -7.26233190e-03]
  [ 1.99861920e+00  1.98867702e+00  1.99550426e+00 ... -7.52283633e-03
   -7.03763403e-03 -1.58877042e-03]]

 [[ 7.21862221e+00  7.28073978e+00  7.28304482e+00 ...  2.53308434e-02
   -8.89940932e-03 -1.70947965e-02]
  [ 7.30912209e+00  7.3